# **ED1 : Prétraitement et Sélection de Caractéristiques sur un Jeu de Données de Fabrication de LEGO**


## 1- Importation et exploration des données

**Importation**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder
import plotly.express as px

df = pd.read_excel("factory_process_CLA_raw.xlsx")
data = df.copy()

**Exploration des proportions des classes dans la variable cible**

In [2]:
# Identifier les variables quantitatives (numériques)
numerical = data.select_dtypes(exclude="object").columns

# Identifier les variables qualitatives (catégorielles)
categorical = data.select_dtypes(include="object").columns

proportion = {}
for col in categorical:
    proportion[col] = data[col].value_counts(normalize=True)

proportion = pd.DataFrame(proportion)
df_melted = proportion.reset_index().melt(id_vars="index",
                                          var_name="Conformity",
                                          value_name="Proportion")
df_melted.rename(columns={"index": "Status"}, inplace=True)

# Stacked bar chart
fig = px.bar(df_melted,
             x="Conformity",
             y="Proportion",
             color="Status",
             barmode="stack",
             title="Conform vs Non-conform per Conformity Check")

fig.show()


## 2- Prétraitement des Données

**Detection des valeurs manquantes**

In [3]:
# Nombre de valeurs manquantes par colonne
missing_counts = data[numerical].isnull().sum()
missing_percent = data[numerical].isnull().mean() * 100
print(pd.concat([missing_counts, missing_percent], axis=1, keys=['Missing', 'Percent']))

                                                    Missing    Percent
Humidity                                                 81  10.012361
Temperature                                              81  10.012361
Machine1,RawMaterial,Property1                           81  10.012361
Machine1,RawMaterial,Property2                           81  10.012361
Machine1,RawMaterial,Property3                           81  10.012361
Machine1,RawMaterial,Property4                           81  10.012361
Machine1,RawMaterialFeederParameter,U,Actual             81  10.012361
Machine1,Zone1Temperature,C,Actual                       81  10.012361
Machine1,Zone2Temperature,C,Actual                       81  10.012361
Machine1,MotorAmperage,U,Actual                          81  10.012361
Machine1,MotorRPM,C,Actual                               81  10.012361
Machine1,MaterialPressure,U,Actual                       81  10.012361
Machine1,MaterialTemperature,U,Actual                    81  10.012361
Machin

**Exploration des distributions numériques**

In [4]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

# ---------------- CONFIG ----------------
# df should already exist in your environment
# numerical can be a pandas.Index or a list of column names
# Example to auto-get numerical:
# numerical = df.select_dtypes(include='number').columns
bw_method = None  # tweak bandwidth if needed, e.g. 0.3
# ----------------------------------------
df=data
numerical = list(numerical)  # make sure it's a simple list
if len(numerical) == 0:
    raise ValueError("`numerical` is empty — provide at least one numeric column name.")

# First pass: compute KDEs for non-constant columns to get a sensible global y-scale
kde_results = {}    # col -> (x_grid, y) for non-constant
constant_info = {}  # col -> (value, count) for constant cols

for col in numerical:
    vals = df[col].dropna().values
    if vals.size == 0:
        # empty column
        continue

    if np.allclose(vals, vals[0]):
        # constant column
        constant_info[col] = (float(vals[0]), int(vals.size))
    else:
        # compute KDE (safe fallback to histogram interp)
        try:
            kde = gaussian_kde(vals, bw_method=bw_method)
            lo, hi = vals.min(), vals.max()
            margin = (hi - lo) * 0.1 if hi > lo else 1.0
            x_grid = np.linspace(lo - margin, hi + margin, 400)
            y = kde(x_grid)
        except Exception:
            # fallback: density from histogram then interp
            lo, hi = vals.min(), vals.max()
            margin = (hi - lo) * 0.1 if hi > lo else 1.0
            x_grid = np.linspace(lo - margin, hi + margin, 400)
            hist, edges = np.histogram(vals, bins=50, density=True)
            bin_centers = (edges[:-1] + edges[1:]) / 2
            y = np.interp(x_grid, bin_centers, hist, left=0, right=0)

        kde_results[col] = (x_grid, y)

# determine a global ymax for plotting constant verticals (based on computed KDEs)
global_ymax = 0.0
for x, y in kde_results.values():
    if y.max() > global_ymax:
        global_ymax = float(y.max())

# if no non-constant columns exist, choose a default ymax
if global_ymax == 0:
    global_ymax = 1.0

# Build traces: KDEs for non-constant, vertical line + marker for constant
traces = []
for col in numerical:
    # empty column -> invisible empty trace
    vals = df[col].dropna().values
    if vals.size == 0:
        traces.append(
            go.Scatter(
                x=[None], y=[None], name=col, visible=False,
                hoverinfo='text', hovertext=f'{col}: no non-null values'
            )
        )
        continue

    if col in kde_results:
        x_grid, y = kde_results[col]
        traces.append(
            go.Scatter(
                x=x_grid,
                y=y,
                mode='lines',
                fill='tozeroy',
                name=col,
                visible=False,
                hovertemplate=f'{col}<br>x: %{{x:.5g}}<br>density: %{{y:.5g}}<extra></extra>'
            )
        )
    else:
        # constant column: draw vertical line and a marker at top with count
        v, cnt = constant_info[col]
        y_top = global_ymax * 0.9  # vertical spike height (90% of KDE max)
        traces.append(
            go.Scatter(
                x=[v, v],
                y=[0, y_top],
                mode='lines',
                line=dict(width=3, dash='dash'),
                name=col,
                visible=False,
                hovertemplate=f'{col}<br>Value: {v:.5g}<br>Count: {cnt}<extra></extra>'
            )
        )
        # marker + annotation point so hover shows a label and it looks visible in legend
        traces.append(
            go.Scatter(
                x=[v],
                y=[y_top],
                mode='markers+text',
                marker=dict(size=8),
                text=[f'n={cnt}'],
                textposition='top center',
                name=f'{col} (constant)',
                visible=False,
                hovertemplate=f'{col}<br>Value: {v:.5g}<br>Count: {cnt}<extra></extra>'
            )
        )

# Make the first non-empty trace visible (find first trace that's not empty None)
first_visible_idx = None
for i, t in enumerate(traces):
    if t.x is not None and not (len(t.x)==1 and t.x[0] is None):
        first_visible_idx = i
        break
if first_visible_idx is not None:
    traces[first_visible_idx].visible = True

# Build dropdown buttons. Note: for constant cols we added 2 traces (line+marker), so we must turn on both
buttons = []
# need mapping from col -> indices in traces to show when selected
col_to_trace_idxs = {}
i = 0
for col in numerical:
    vals = df[col].dropna().values
    if vals.size == 0:
        col_to_trace_idxs[col] = [i]
        i += 1
        continue
    if col in kde_results:
        col_to_trace_idxs[col] = [i]
        i += 1
    else:
        # constant column used two traces (line + marker)
        col_to_trace_idxs[col] = [i, i+1]
        i += 2

for col in numerical:
    visible = [False] * len(traces)
    for idx in col_to_trace_idxs.get(col, []):
        visible[idx] = True
    buttons.append(dict(
        label=col,
        method='update',
        args=[{'visible': visible},
              {'title': f'KDE / Reality: {col}',
               'yaxis': {'title': 'Density (or spike)'},
               'xaxis': {'title': col}}]
    ))

fig = go.Figure(data=traces)
fig.update_layout(
    title=f'KDE / Reality: {numerical[0]}',
    updatemenus=[dict(active=0, buttons=buttons, x=0.0, y=1.12, xanchor='left', yanchor='top')],
    margin=dict(t=100, b=50, l=50, r=20),
    xaxis=dict(title=numerical[0]),
    yaxis=dict(title='Density (or spike)'),
    height=520,
)

fig.show()


**2.1- Gestion des valeurs manquantes**

Pour chaque variable numérique, nous choisissons la méthode d’imputation la plus adaptée selon sa distribution et la présence d’outliers :

- **Analyse de la skewness (asymétrie)** : si la distribution est approximativement symétrique (`|skew| < 0.5`)  
- **Analyse des outliers** : si aucun outlier n’est détecté via la règle IQR  

**Règles choisies :**  
- Si distribution symétrique et pas d’outliers → imputation par la **moyenne**  
- Sinon → imputation par la **médiane**  

Chaque colonne est ensuite imputée selon la méthode correspondante, assurant une gestion robuste des valeurs manquantes.


In [5]:
# Créer dictionnaire pour stocker la méthode choisie
impute_methods = {}

for col in numerical:
    col_skew = data[col].skew()
    # Calcul IQR
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = data[(data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)]

    # Choix de la méthode
    if abs(col_skew) < 0.5 and len(outliers) == 0:
        impute_methods[col] = 'mean'
    else:
        impute_methods[col] = 'median'

# Appliquer l’imputation
for col, method in impute_methods.items():
    imputer = SimpleImputer(strategy=method)
    data[[col]] = imputer.fit_transform(data[[col]])

print(impute_methods)

{'Humidity': 'mean', 'Temperature': 'median', 'Machine1,RawMaterial,Property1': 'mean', 'Machine1,RawMaterial,Property2': 'mean', 'Machine1,RawMaterial,Property3': 'mean', 'Machine1,RawMaterial,Property4': 'mean', 'Machine1,RawMaterialFeederParameter,U,Actual': 'median', 'Machine1,Zone1Temperature,C,Actual': 'median', 'Machine1,Zone2Temperature,C,Actual': 'mean', 'Machine1,MotorAmperage,U,Actual': 'median', 'Machine1,MotorRPM,C,Actual': 'median', 'Machine1,MaterialPressure,U,Actual': 'median', 'Machine1,MaterialTemperature,U,Actual': 'median', 'Machine1,ExitZoneTemperature,C,Actual': 'median', 'Machine2,RawMaterial,Property1': 'median', 'Machine2,RawMaterial,Property2': 'median', 'Machine2,RawMaterial,Property3': 'median', 'Machine2,RawMaterial,Property4': 'median', 'Machine2,RawMaterialFeederParameter,U,Actual': 'median', 'Machine2,Zone1Temperature,C,Actual': 'median', 'Machine2,Zone2Temperature,C,Actual': 'median', 'Machine2,MotorAmperage,U,Actual': 'median', 'Machine2,MotorRPM,C,Act

In [6]:
data[numerical].sample(20)

,Humidity,Temperature,"Machine1,RawMaterial,Property1","Machine1,RawMaterial,Property2","Machine1,RawMaterial,Property3","Machine1,RawMaterial,Property4","Machine1,RawMaterialFeederParameter,U,Actual","Machine1,Zone1Temperature,C,Actual","Machine1,Zone2Temperature,C,Actual","Machine1,MotorAmperage,U,Actual",...,"Machine3,Zone1Temperature,C,Actual","Machine3,Zone2Temperature,C,Actual","Machine3,MotorAmperage,U,Actual","Machine3,MotorRPM,C,Actual","Machine3,MaterialPressure,U,Actual","Machine3,MaterialTemperature,U,Actual","Machine3,ExitZoneTemperature,C,Actual","FirstStage,CombinerOperation,Temperature1,U,Actual","FirstStage,CombinerOperation,Temperature2,U,Actual","FirstStage,CombinerOperation,Temperature3,C,Actual"
227,16.950000,23.86,11.54,200.0,963.0,247.0,1264.230000,72.3,71.400000,76.820000,...,78.0,77.9,338.32,13.62,241.240,69.2,64.900000,104.70000,69.700000,79.8
350,16.950000,23.86,11.54,200.0,963.0,247.0,1257.630000,72.1,71.900000,76.280000,...,78.0,78.0,338.03,13.76,240.510,69.6,65.100000,105.80000,69.600000,80.0
360,16.950000,23.86,11.54,200.0,963.0,247.0,1258.240000,72.1,72.014045,78.440000,...,78.0,77.9,339.54,13.53,240.520,69.8,65.033929,105.90000,70.100000,80.0
643,16.730000,24.09,11.54,200.0,963.0,247.0,1262.580000,72.1,71.400000,75.740000,...,78.0,78.0,341.36,13.63,238.690,71.4,65.000000,106.10000,69.300000,79.9
348,16.950000,23.86,11.54,200.0,963.0,247.0,1266.489392,72.1,72.021024,77.897555,...,78.0,78.0,337.54,13.62,240.105,69.6,65.100000,105.26588,69.848695,80.0
285,16.950000,23.86,11.54,200.0,963.0,247.0,1274.230000,72.2,72.600000,76.150000,...,78.0,77.9,339.12,13.93,240.540,69.4,65.033929,105.40000,69.400000,80.2
4,17.240000,23.53,11.54,200.0,963.0,247.0,1247.260000,72.1,72.700000,49.100000,...,78.1,78.3,341.41,13.47,252.130,65.8,65.100000,103.40000,111.600000,80.0
653,16.730000,24.09,11.54,200.0,963.0,247.0,1262.580000,72.1,71.400000,75.210000,...,78.0,78.0,345.78,13.81,238.920,69.8,65.000000,106.20000,68.900000,79.8
546,16.730000,24.00,11.54,200.0,963.0,247.0,1272.110000,72.0,71.800000,76.820000,...,78.0,77.9,343.71,13.70,238.620,69.8,65.000000,106.50000,69.600000,79.8
557,16.730000,24.00,11.54,200.0,963.0,247.0,1262.580000,72.0,72.000000,76.690000,...,78.0,78.0,339.54,13.68,238.210,71.1,65.033929,106.70000,66.800000,80.0


**2.2- Encodage des Variables Qualitatives et Vérification des encodages**

Pour préparer les variables catégorielles à l'analyse et à la modélisation, nous procédons à un **encodage binaire manuel** :  

- `"Conform"` → `1`  
- `"Non-conform"` → `0`  

Cette méthode simple permet de **garder une notation cohérente** pour toutes les variables cibles.  
Nous ne passons pas par un `LabelEncoder` classique, car celui-ci pourrait attribuer des codes différents selon la variable ou l’ordre des catégories, ce qui compliquerait la comparaison entre cibles.


In [7]:
data[categorical] = data[categorical].replace({"Conform":1, "Non-conform":0})
data[categorical].sample(20)

/tmp/ipython-input-30743468.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Conformity1,Conformity2,Conformity3,Conformity4,Conformity5,Conformity6,Conformity7,Conformity8,Conformity9,Conformity10,Conformity11,Conformity12
507,1,0,1,1,1,0,1,1,1,1,1,1
688,1,0,1,1,1,0,1,1,1,1,1,1
409,1,0,1,1,1,0,1,1,1,1,1,1
771,1,1,1,1,1,0,1,1,1,1,1,1
548,0,0,1,1,1,0,1,1,1,1,1,1
62,0,1,1,1,1,0,1,1,0,1,1,1
769,1,1,1,1,1,0,1,1,1,1,1,1
151,1,1,1,1,1,0,1,1,1,1,1,1
345,1,0,1,1,1,0,1,1,1,1,1,1
602,1,0,1,1,1,0,1,1,1,1,1,1


**2.3- Mise à l'Échelle des Données :**

La mise à l'échelle (standardisation) permet de centrer les variables autour de 0 et de les ramener à une variance unité.  
Elle est essentielle pour comparer correctement les variables lors des méthodes de sélection de caractéristiques.  
Elle garantit aussi que certains algorithmes (ex. k-NN, SVM, régressions réguliarisées) ne soient pas biaisés par des variables à grande échelle.  
Ainsi, toutes les variables contribuent de manière équitable à l'analyse et à l'apprentissage.


In [8]:
from sklearn.preprocessing import StandardScaler

# Initialiser le StandardScaler
scaler = StandardScaler()

# Ajuster et transformer les données
data_scaled = scaler.fit_transform(data[numerical])

data[numerical] = data_scaled
data[numerical].sample(20)

,Humidity,Temperature,"Machine1,RawMaterial,Property1","Machine1,RawMaterial,Property2","Machine1,RawMaterial,Property3","Machine1,RawMaterial,Property4","Machine1,RawMaterialFeederParameter,U,Actual","Machine1,Zone1Temperature,C,Actual","Machine1,Zone2Temperature,C,Actual","Machine1,MotorAmperage,U,Actual",...,"Machine3,Zone1Temperature,C,Actual","Machine3,Zone2Temperature,C,Actual","Machine3,MotorAmperage,U,Actual","Machine3,MotorRPM,C,Actual","Machine3,MaterialPressure,U,Actual","Machine3,MaterialTemperature,U,Actual","Machine3,ExitZoneTemperature,C,Actual","FirstStage,CombinerOperation,Temperature1,U,Actual","FirstStage,CombinerOperation,Temperature2,U,Actual","FirstStage,CombinerOperation,Temperature3,C,Actual"
717,-2.464542,2.092227,3.552714e-15,0.0,0.0,0.0,0.424678,-0.755299,-7.225638e-01,0.401762,...,-0.462837,0.534700,-0.666208,-1.233687,-0.933749,1.333794,-4.436278e-01,8.989028e-01,-0.419758,-0.970587
571,-0.968053,0.723609,3.552714e-15,0.0,0.0,0.0,0.156216,-0.755299,6.579343e-01,0.506532,...,-0.462837,0.534700,0.655415,-0.886249,-0.622773,0.719839,-1.751163e+00,1.764972e+00,-0.596825,1.673201
70,1.663702,-1.750430,3.552714e-15,0.0,0.0,0.0,0.256759,0.029087,-2.623977e-01,0.483153,...,-0.462837,1.334342,-0.239652,-0.422999,1.851271,-1.684815,8.639077e-01,-9.294649e-01,2.118205,-0.089325
178,1.663702,-1.750430,3.552714e-15,0.0,0.0,0.0,-0.487028,0.029087,-3.269676e-14,-0.285737,...,1.384704,-1.064586,2.862782,-0.422999,0.783497,-1.122023,2.171443e+00,-4.483155e-01,2.094596,0.791938
423,0.167214,-0.013339,3.552714e-15,0.0,0.0,0.0,0.205563,0.029087,-9.526468e-01,0.587923,...,-0.462837,-0.264943,-1.547290,1.545815,-0.176950,0.003559,-4.436278e-01,1.290638e-01,-0.390247,-0.089325
20,0.000000,-1.750430,3.552714e-15,0.0,0.0,0.0,0.012104,2.382246,1.118100e+00,0.483153,...,2.616397,2.133985,-0.875989,-0.307186,2.632838,-1.991792,8.639077e-01,-1.603074e+00,2.023769,0.791938
694,0.000000,1.197362,3.552714e-15,0.0,0.0,0.0,0.205563,-0.755299,-3.231474e-02,0.541167,...,-0.462837,-1.864229,0.923469,0.156064,-0.713589,0.873328,-4.436278e-01,9.951327e-01,-0.431562,1.673201
762,-0.968053,1.197362,3.552714e-15,0.0,0.0,0.0,0.199322,-0.755299,-1.412813e+00,0.133343,...,0.153010,1.334342,-0.020547,0.040252,-1.352052,1.845422,-4.436278e-01,-1.314384e+00,-0.797501,-0.089325
31,1.663702,-1.750430,3.552714e-15,0.0,0.0,0.0,0.031173,2.382246,8.880173e-01,-1.987167,...,3.848091,2.133985,-1.773387,0.503502,2.484230,0.003559,8.639077e-01,-1.367509e-14,2.053280,-0.089325
597,-0.968053,0.723609,3.552714e-15,0.0,0.0,0.0,0.165808,-0.755299,1.118100e+00,0.447653,...,-0.462837,-0.264943,-0.451764,0.040252,-0.578741,0.003559,-4.436278e-01,1.380052e+00,-0.466976,-2.733112


## 3- Sélection de Caractéristiques

**3.1- Méthode de corrélation**

Pour cette méthode nous avons commencé par tracer une heatmap pour regarder les corrélations entre nos variables quantitatives, il s'avère qu'il y en a plusieurs qui sont corrélés linéairement en eux.

In [9]:
import plotly.express as px

fig = px.imshow(data[numerical].corr(), width=1500, height=1500, text_auto=True, color_continuous_scale="RdYlGn")
fig.show()


#### Réduction de variables numériques par Corrélation

#### Définition
La **corrélation** mesure le degré de dépendance linéaire entre deux variables numériques.  
Lorsqu’elle est élevée (|r| ≥ seuil), cela signifie que les deux variables transportent une information similaire → on en supprime une pour réduire la redondance.

---

#### Étapes
- **Préparation**
  - Fixer un seuil (ex. 0.7).
  - Supprimer les colonnes constantes.
- **Calcul**
  - Construire la matrice de corrélation absolue `M_corr`.
  - Identifier, pour chaque variable, le nombre de corrélations > seuil.
- **Sélection**
  - Supprimer progressivement la variable la plus corrélée.
  - Générer la liste `correlation_selected`.

---

#### Sorties
- **Matrice `M_corr`** : corrélations absolues entre toutes les variables numériques.  
- **Liste `correlation_selected`** : variables numériques retenues après suppression des redondantes.  
- **Résumé** : nombre de colonnes supprimées et conservées.


In [10]:
# seuil de corrélation
seuil = 0.7

# suppression des colonnes constantes
const_cols = [col for col in numerical if data[col].var() == 0]

# matrice de corrélation
M_corr = data[numerical].corr().abs()

# on met la diagonale à 0 pour ignorer la corrélation avec soi-même
np.fill_diagonal(M_corr.values, 0)

# colonnes à supprimer
colonnes_supprimer = set(const_cols)

# liste des colonnes à traiter
cols = set(numerical) - set(const_cols)

while cols:
    cols_list = list(cols)  # <-- important : convert set en list pour pandas
    # compter combien de corrélations > seuil pour chaque colonne
    corr_count = (M_corr.loc[cols_list, cols_list] > seuil).sum()
    # trouver la colonne la plus corrélée
    if corr_count.max() == 0:
        break
    col_to_remove = corr_count.idxmax()
    colonnes_supprimer.add(col_to_remove)
    cols.remove(col_to_remove)

# mise à jour de la liste numerical
correlation_selected = [col for col in numerical if col not in colonnes_supprimer]
cols_reduits = set(colonnes_supprimer).union(set(const_cols))
print(f"Colonnes supprimées ({len(cols_reduits)}):", cols_reduits)
print(f"Colonnes sélectionnées ({len(correlation_selected)}):", correlation_selected)

Colonnes supprimées (16): {'Machine3,RawMaterial,Property4', 'Machine3,MaterialTemperature,U,Actual', 'Machine1,RawMaterial,Property2', 'Machine2,RawMaterial,Property1', 'Machine1,RawMaterial,Property4', 'Temperature', 'Machine3,MaterialPressure,U,Actual', 'Machine2,RawMaterial,Property4', 'FirstStage,CombinerOperation,Temperature2,U,Actual', 'Machine1,MaterialTemperature,U,Actual', 'Machine1,MotorAmperage,U,Actual', 'Machine1,RawMaterial,Property3', 'Machine1,RawMaterial,Property1', 'Machine3,RawMaterial,Property2', 'Machine2,MaterialTemperature,U,Actual', 'Machine2,RawMaterial,Property2'}
Colonnes sélectionnées (25): ['Humidity', 'Machine1,RawMaterialFeederParameter,U,Actual', 'Machine1,Zone1Temperature,C,Actual', 'Machine1,Zone2Temperature,C,Actual', 'Machine1,MotorRPM,C,Actual', 'Machine1,MaterialPressure,U,Actual', 'Machine1,ExitZoneTemperature,C,Actual', 'Machine2,RawMaterial,Property3', 'Machine2,RawMaterialFeederParameter,U,Actual', 'Machine2,Zone1Temperature,C,Actual', 'Machin

**3.2- Méthode de sélection univariée**

#### Sélection de caractéristiques numériques par ANOVA

#### Définition
L’**ANOVA (Analyse de la Variance)** compare la variance intra-groupes et inter-groupes d’une variable numérique en fonction d’une variable catégorielle.  
Elle permet de savoir si une variable numérique discrimine significativement les classes d’une variable cible.

---

#### Étapes
- **Préparation**
  - Supprimer les variables constantes.
  - Définir un seuil `seuil_p` (ex. 0.05).
- **Calcul**
  - Pour chaque couple `(variable numérique, variable catégorielle)` :
    - Séparer les observations en groupes selon la variable cible.
    - Appliquer le test `f_oneway`.
    - Stocker la p-value dans `anova_results`.
- **Sélection**
  - Conserver les variables numériques dont la p-value < `seuil_p` pour au moins une cible.
  - Générer la liste `anova_selected`.

---

#### Sorties
- **Matrice `anova_results`** : p-values des tests ANOVA.  
- **Liste `anova_selected`** : variables numériques discriminantes.  
- **Visualisation** : heatmap interactive (Plotly) des p-values.


In [11]:
from scipy.stats import f_oneway

#selectionner les valeurs non constantes
numerical_nonconst = [col for col in numerical if data[col].nunique() > 1]
# Seuil de p-value pour considérer la variable comme significative
seuil_p = 0.05

anova_results = pd.DataFrame(index=numerical_nonconst, columns=categorical)

# Boucle sur chaque variable cible catégorielle
for cat_col in categorical:
    for num_col in numerical_nonconst:
        # Extraire les valeurs numériques pour chaque classe
        groups = [data[num_col][data[cat_col] == cls].dropna()
                  for cls in data[cat_col].unique()]

        # Test ANOVA à un facteur
        if len(groups) > 1:  # ANOVA nécessite au moins 2 groupes
            f_stat, p_val = f_oneway(*groups)
            anova_results.loc[num_col, cat_col] = p_val
        else:
            anova_results.loc[num_col, cat_col] = None

# Conversion en float
anova_results = anova_results.astype(float)

# Sélection des variables significatives selon le seuil
anova_selected = anova_results.index[(anova_results < seuil_p).any(axis=1)].tolist()

print(f"Colonnes supprimées ({len(set(numerical)-set(anova_selected))}):", set(numerical)-set(anova_selected))
print(f"Colonnes sélectionnées ({len(anova_selected)}):", anova_selected)
# Heatmap interactive des p-values ANOVA
fig = px.imshow(anova_results,
                width=1000, height=800,
                text_auto=".3f",
                color_continuous_scale="Viridis",
                title="P-values ANOVA : variables numériques vs cibles catégorielles")

fig.update_layout(xaxis_title="Variables cibles", yaxis_title="Variables numériques")
fig.show()


Colonnes supprimées (10): {'Machine3,RawMaterial,Property4', 'Machine1,RawMaterial,Property3', 'Machine3,RawMaterial,Property1', 'Machine1,RawMaterial,Property2', 'Machine2,MotorRPM,C,Actual', 'Machine1,RawMaterial,Property1', 'Machine3,RawMaterial,Property3', 'Machine3,RawMaterial,Property2', 'Machine1,RawMaterial,Property4', 'Machine1,MotorRPM,C,Actual'}
Colonnes sélectionnées (31): ['Humidity', 'Temperature', 'Machine1,RawMaterialFeederParameter,U,Actual', 'Machine1,Zone1Temperature,C,Actual', 'Machine1,Zone2Temperature,C,Actual', 'Machine1,MotorAmperage,U,Actual', 'Machine1,MaterialPressure,U,Actual', 'Machine1,MaterialTemperature,U,Actual', 'Machine1,ExitZoneTemperature,C,Actual', 'Machine2,RawMaterial,Property1', 'Machine2,RawMaterial,Property2', 'Machine2,RawMaterial,Property3', 'Machine2,RawMaterial,Property4', 'Machine2,RawMaterialFeederParameter,U,Actual', 'Machine2,Zone1Temperature,C,Actual', 'Machine2,Zone2Temperature,C,Actual', 'Machine2,MotorAmperage,U,Actual', 'Machine2,

**3.3- Méthode d'information mutuelle**

#### Sélection de caractéristiques par Information Mutuelle

#### Définition
L’**information mutuelle (MI)** mesure la quantité d’information partagée entre deux variables.  
Elle indique dans quelle mesure la connaissance d’une variable réduit l’incertitude sur l’autre.  
En sélection de caractéristiques, on l’utilise pour détecter les variables numériques les plus informatives vis-à-vis des variables cibles catégorielles, même en présence de relations **non-linéaires**.

---

#### Étapes
- **Préparation**
  - Prendre les variables numériques `numerical` et les variables catégorielles `categorical`.
  - Fixer un seuil `seuil_mi` (ex. 0.03).
- **Calcul**
  - Pour chaque variable cible catégorielle `y` :
    - Calculer `mutual_info_classif(X, y)` où `X = data[numerical]`.
    - Stocker les scores MI dans une matrice `mutual_info_scores`.
- **Sélection**
  - Conserver les variables numériques dont au moins un score MI ≥ `seuil_mi`.
  - Générer la liste `mi_selected`.

---

#### Sorties
- **Matrice `mutual_info_scores`** : tableau des scores MI pour chaque couple `(variable numérique, variable cible)`.  
- **Liste `mi_selected`** : variables numériques jugées significatives (MI ≥ seuil).  
- **Résumé** :  
  - Nombre de colonnes supprimées.  
  - Nombre de colonnes retenues.  
- **Visualisation** : graphique en barres (Plotly) illustrant la contribution de chaque variable.


In [14]:
from sklearn.feature_selection import mutual_info_classif
seuil_mi = 0.03
mutual_info_scores = pd.DataFrame(index=numerical, columns=categorical)

for col_target in categorical:
    y = data[col_target]
    X = data[numerical]

    # Calcul de l'information mutuelle
    mi = mutual_info_classif(X, y, discrete_features=False, random_state=42)

    mutual_info_scores[col_target] = mi

# Afficher les scores
mutual_info_scores = mutual_info_scores.astype(float)
#print(mutual_info_scores.sort_values(by=categorical[0], ascending=False))
mi_selected = mutual_info_scores.index[(mutual_info_scores >= seuil_mi).any(axis=1)].tolist()
print(f"Colonnes supprimées ({len(set(numerical)-set(mi_selected))}):", set(numerical)-set(mi_selected))
print(f"Colonnes sélectionnées ({len(mi_selected)}):", mi_selected)

fig = px.bar(mutual_info_scores.reset_index().melt(id_vars='index'),width=1200, height=600, x='index', y='value', color='variable', barmode='group', title="Information Mutuelle entre variables quantitatives et cibles qualitatives")
fig.show()


Colonnes supprimées (12): {'Machine3,RawMaterial,Property4', 'Machine1,RawMaterial,Property3', 'Machine3,RawMaterial,Property1', 'Machine1,RawMaterial,Property2', 'Machine1,RawMaterial,Property1', 'Machine3,MotorAmperage,U,Actual', 'Machine3,RawMaterial,Property3', 'Machine3,RawMaterial,Property2', 'Machine3,MotorRPM,C,Actual', 'Machine1,RawMaterial,Property4', 'Machine2,ExitZoneTemperature,C,Actual', 'Machine1,MotorRPM,C,Actual'}
Colonnes sélectionnées (29): ['Humidity', 'Temperature', 'Machine1,RawMaterialFeederParameter,U,Actual', 'Machine1,Zone1Temperature,C,Actual', 'Machine1,Zone2Temperature,C,Actual', 'Machine1,MotorAmperage,U,Actual', 'Machine1,MaterialPressure,U,Actual', 'Machine1,MaterialTemperature,U,Actual', 'Machine1,ExitZoneTemperature,C,Actual', 'Machine2,RawMaterial,Property1', 'Machine2,RawMaterial,Property2', 'Machine2,RawMaterial,Property3', 'Machine2,RawMaterial,Property4', 'Machine2,RawMaterialFeederParameter,U,Actual', 'Machine2,Zone1Temperature,C,Actual', 'Machin

**3.4- Test de chi2**

#### Réduction de variables catégorielles par le test Khi²

#### Définition
Le **test Khi² d’indépendance** évalue si deux variables catégorielles sont statistiquement indépendantes.  
En sélection de caractéristiques, il permet de **supprimer les variables redondantes** (fortement dépendantes entre elles) et de conserver uniquement les plus représentatives.

---

#### Étapes
- **Préparation**
  - Éliminer les variables trop déséquilibrées (≥ 95% d’une même modalité).
  - Construire la liste `categorical` filtrée.
- **Calcul**
  - Pour chaque paire de variables catégorielles :
    - Construire la table de contingence (`pd.crosstab`).
    - Appliquer le test `chi2_contingency`.
    - Stocker la p-value dans une matrice `pval_matrix`.
- **Sélection**
  - Si `p < 0.05` → dépendance significative :
    - Supprimer la variable la moins informative (variance la plus faible).
  - Conserver la liste `categorical_chi2_selected`.

---

#### Sorties
- **Matrice `pval_matrix`** : p-values du test Khi² entre toutes les paires de variables catégorielles.  
- **Liste `categorical_chi2_selected`** : variables catégorielles indépendantes retenues.  
- **Visualisation** : heatmap interactive (Plotly) des dépendances entre variables.


In [13]:
from scipy.stats import chi2_contingency

to_drop = set()
seuil = 0.95
categorical = [col for col in categorical if data[col].value_counts(normalize=True).max() <= seuil]
print("Colonnes catégorielles conservées (moins de 95% d'une même modalité) :", categorical)
pval_matrix = pd.DataFrame(index=categorical, columns=categorical)

# Boucle sur chaque paire de colonnes
for i in range(len(categorical)):
    for j in range(i + 1, len(categorical)):
        col1 = categorical[i]
        col2 = categorical[j]

        # Tableau de contingence
        contingency = pd.crosstab(data[col1], data[col2])

        # Test Khi²
        chi2, p, dof, expected = chi2_contingency(contingency)

        pval_matrix.loc[col1, col2] = p

        # Si l’une des colonnes a déjà été supprimée, ignorer
        if col1 in to_drop or col2 in to_drop:
            continue

        # Si dépendance significative, supprimer col2
        if p < 0.05:
            # Calculer la variance de chaque colonne
            var1 = data[col1].var()
            var2 = data[col2].var()

            # Supprimer la colonne avec la plus petite variance
            if var1 >= var2:
                to_drop.add(col2)
            else:
                to_drop.add(col1)

# Supprimer les colonnes supprimées de la liste categorical
categorical_chi2_selected = [col for col in categorical if col not in to_drop]


# Affichage des résultats
print("Colonnes filtrée indépendantes :", categorical_chi2_selected)

# Convertir en float
pval_matrix = pval_matrix.astype(float)

# Heatmap interactive avec Plotly
fig = px.imshow(pval_matrix, width=800, height=800, text_auto=".2f", color_continuous_scale=["red", "yellow", "green"], title="Dépendance entre variables catégorielles (p-values Khi²)")


fig.update_layout(xaxis_title="Variables", yaxis_title="Variables")
fig.show()



Colonnes catégorielles conservées (moins de 95% d'une même modalité) : ['Conformity1 ', 'Conformity2', 'Conformity4', 'Conformity9', 'Conformity10']
Colonnes filtrée indépendantes : ['Conformity2', 'Conformity4']


## 4- Comparaison des Méthodes et Réduction de Dimension

**4.1- Réduction variables quantitatives**

#### *Définition*
La réduction des variables quantitatives consiste à ne retenir que celles jugées pertinentes par plusieurs méthodes de sélection de caractéristiques.  
L’objectif est d’augmenter la **robustesse** du choix en croisant les résultats de différentes approches statistiques : corrélation, ANOVA et information mutuelle.

#### *Étapes*
- Appliquer indépendamment chaque méthode de sélection (Corrélation, ANOVA, Information Mutuelle).  
- Obtenir pour chaque méthode une liste des variables retenues.  
- Construire un tableau de comparaison binaire indiquant, pour chaque variable :  
  - `1` si la méthode l’a sélectionnée  
  - `0` sinon  
- Calculer pour chaque variable le **nombre total de méthodes** qui l’ont sélectionnée (`count`).  
- Fixer un seuil de sélection basé sur ce consensus :  
  - Garder les variables présentes dans **les 3 méthodes** (`count = 3`).  

#### *Sorties*
- Un tableau de comparaison binaire montrant les résultats des trois méthodes.  
- Une liste réduite de variables quantitatives robustes, sélectionnées par consensus.  
- Un jeu de données final épuré des variables peu pertinentes, facilitant la modélisation et l’analyse ultérieure.


In [25]:

# listes uniques
A = list(dict.fromkeys(correlation_selected))
B = list(dict.fromkeys(anova_selected))
C = list(dict.fromkeys(mi_selected))

# toutes les colonnes numériques (base complète de comparaison)
all_vars = list(df[numerical].columns)

# tableau binaire de comparaison
df_bin = pd.DataFrame({'variable': all_vars})

df_bin['Correlation'] = df_bin['variable'].isin(A).astype(int)
df_bin['ANOVA'] = df_bin['variable'].isin(B).astype(int)
df_bin['Mutual_Info'] = df_bin['variable'].isin(C).astype(int)

# nombre de méthodes qui l’ont choisie
df_bin['count'] = df_bin[['Correlation','ANOVA','Mutual_Info']].sum(axis=1)

# trier pour voir d’abord les variables communes
df_bin = df_bin.sort_values(by=['count','variable'], ascending=[False, True]).reset_index(drop=True)

df_bin


,variable,Correlation,ANOVA,Mutual_Info,count
0,"FirstStage,CombinerOperation,Temperature1,U,Ac...",1,1,1,3
1,"FirstStage,CombinerOperation,Temperature3,C,Ac...",1,1,1,3
2,Humidity,1,1,1,3
3,"Machine1,ExitZoneTemperature,C,Actual",1,1,1,3
4,"Machine1,MaterialPressure,U,Actual",1,1,1,3
5,"Machine1,RawMaterialFeederParameter,U,Actual",1,1,1,3
6,"Machine1,Zone1Temperature,C,Actual",1,1,1,3
7,"Machine1,Zone2Temperature,C,Actual",1,1,1,3
8,"Machine2,MaterialPressure,U,Actual",1,1,1,3
9,"Machine2,MotorAmperage,U,Actual",1,1,1,3


In [29]:
# Variables retenues par les 3 méthodes
selected_vars = df_bin.loc[df_bin['count'] == 3, 'variable'].tolist()

pd.DataFrame({"Selected Variables":selected_vars})

,Selected Variables
0,"FirstStage,CombinerOperation,Temperature1,U,Ac..."
1,"FirstStage,CombinerOperation,Temperature3,C,Ac..."
2,Humidity
3,"Machine1,ExitZoneTemperature,C,Actual"
4,"Machine1,MaterialPressure,U,Actual"
5,"Machine1,RawMaterialFeederParameter,U,Actual"
6,"Machine1,Zone1Temperature,C,Actual"
7,"Machine1,Zone2Temperature,C,Actual"
8,"Machine2,MaterialPressure,U,Actual"
9,"Machine2,MotorAmperage,U,Actual"


**4.2- Réduction des variables qualitatives:**

Pour les variables qualitatives, nous retenons directement les résultats de la méthode **Chi²**.  
Seules les variables catégorielles indépendantes selon le test Chi² (p-value ≥ 0.05) sont conservées, ce qui permet de supprimer les colonnes fortement dépendantes ou peu informatives.


In [30]:
categorical_chi2_selected

['Conformity2', 'Conformity4']

In [32]:
Data_final = data[selected_vars + categorical_chi2_selected]
Data_final.to_excel("Data_final.xlsx")